In [23]:
import polars as pl
import altair as alt

In [24]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [25]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).join(
    pl.scan_parquet(
        "data/projevy_jazyky.parquet",
    ),
    how="left",
    on="text"
).filter(
    pl.col("jazyk") != "sk"
).collect()

In [35]:
df.with_columns(
    pl.col("text").str.extract_all(r"(?i)připadám si \w{0,10}\s{0,1}jako v{0,1}e{0,1}\s{0,1}n{0,1}a{0,1}\s{0,1}\w{1,20}").alias("pripadam_si")
).explode(
    'pripadam_si'
).with_columns(
    pl.col('pripadam_si').str.split(' ').list.slice(-1,1)
).explode(
    'pripadam_si'
).group_by(
    'pripadam_si'
).len().sort(
    by="len",descending=True
)

pripadam_si,len
str,u32
null,699188
"""středověku""",2
"""u""",1
"""učitelka""",1
"""dinosaurus""",1
"""při""",1
"""narkobaron""",1
"""římský""",1
"""největší""",1


In [33]:
df.with_columns(
    pl.col("text").str.extract_all(r"(?i)je mi (to|z toho|teď){0,1}\s{0,1}\w{1,20}").alias("je_mi")
).explode(
    'je_mi'
).with_columns(
    pl.col('je_mi').str.split(' ').list.slice(-1,1)
).explode(
    'je_mi'
).group_by(
    'je_mi'
).len().sort(
    by="len",descending=True
)

je_mi,len
str,u32
null,694476
"""líto""",1448
"""známo""",557
"""jasné""",387
"""velmi""",341
"""velice""",166
"""jedno""",125
"""úplně""",113
"""opravdu""",86
